## 说明

该方法与常见基于北向资金的方法不同，它将价格因素纳入考虑

当然期望能比普通的基于北向的方法好一点，但不能确定，尤其是面对目前的市场，北向卖飞的比比皆是

基于北向的策略在17年以后进行回测天生就有未来数据，因为北向资金塑造A股是目前的共识，所以用北向的数据就等于是用“未来知识”进行测试，高于大盘走势很正常。

在市场风格已经被北向的偏好“极端化”的现在，未来跟随北向的策略是否有效很不确定

本策略将以普通的北向跟踪策略作为比较基准




### 共有4种情况

|     | volume-up  | volume-down |
|  ----  | ----  | ---- |
| price-up  | 1.常见情况 | 2.瞎炒or港资卖飞 |
| price-down  | 3.错杀or港资瞎买 | 4.被抛弃  |

将目标股票设置为<b>当前持仓成本-现价 </b>尽可能大

所以最优先的是情况3，然后是<b>量价齐升</b>，即情况1，再次是情况2，要尽可能避免情况4

### 本文结论的4种情况
|     | 净买入为正  | 净买入为负 |
|  ----  | ----  | ---- |
| 持有量上升  | 结果为正，正常情况<br>可进行正常判断 | 结果为负，异常情况<br>期限内出现极度高卖低买情况 |
| 持有量下降  | 结果为负，异常情况<br>期限内出现极度高买低卖情况 | 结果为正，正常情况<br>比值为净卖出平均价格<br>不具备比较价值  |

其中，仅有左上角的情况是可用的，可能会出现平均价格高于or低于现价的情况，选择成本高于现价最多的即可

## 本例中的4种情况


### 难点
- 对复权的处理，

In [1]:
import pandas as pd
from jqdata import  get_trade_days
# 获取过去一年，每日北向持仓数据
# output:df1：code是col[0]，col为date
def get_vol_roll(end_date, n_days, securities):
    
    if isinstance(end_date, str):
        end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    
    df1 = pd.DataFrame( {'code': securities})
    for ii in range(n_days):
        curr_date = (end_date + datetime.timedelta(-ii)).strftime("%Y-%m-%d")
        
        q1 = query(finance.STK_HK_HOLD_INFO.code, finance.STK_HK_HOLD_INFO.share_number
                  ).filter(finance.STK_HK_HOLD_INFO.day == curr_date, finance.STK_HK_HOLD_INFO.code.in_(securities)
                          ).limit(4000)
        df = finance.run_query(q1)
        # print(df)
        df.rename(columns={'share_number':curr_date}, inplace = True)
        
        if(df.shape[0] == 0):
            continue
        
        df1 = df1.merge(df, on = 'code')
    
    df1.index = df1['code']
    df1.drop(['code'], axis = 1, inplace = True)
    
    return df1

def get_price_roll(end_date, n_days, securities):
    
    if isinstance(end_date, str):
        end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d")
    
    df2 = pd.DataFrame( {'code': securities})
    
    for ii in range(n_days):
        # print('ii',ii)
        curr_date = (end_date + datetime.timedelta(-ii)).strftime("%Y-%m-%d")  
        
        if (get_trade_days(end_date = curr_date, count = 1)[0].toordinal() !=
            datetime.datetime.strptime(curr_date,"%Y-%m-%d").toordinal()):
            continue
        
        df = get_price(securities, start_date=curr_date, end_date=curr_date, frequency='daily', 
              fields=['avg'], skip_paused=False, fq=None, panel=False)
        
        # df4.rename(columns = {'avg':'start_close'}, inplace = True)
        #print(df)
        #assert(df['time'].drop_duplicates().shape[0] == 1)
        #assert(df['time'].drop_duplicates()[0].strftime("%Y-%m-%d") == curr_date)
        
        df = df.drop('time',1)
        #print(df)
        df.rename(columns={'avg':curr_date}, inplace = True)
        #print(df)
        #print(isinstance(df.iloc[0,0],str))
        df2 = df2.merge(df, on = 'code')
        
    df2.index = df2['code']
    df2.drop(['code'], axis = 1, inplace = True)
        
    return df2

In [3]:
import numpy as np
from jqdata import finance

'''
    注意：query得到的ratio都是每10股送转股的数量，要做对应处理
    
'''
def get_expand(code_hold, start_date, end_date):
    
    q = query(finance.STK_XR_XD.code, finance.STK_XR_XD.a_bonus_date,
              finance.STK_XR_XD.dividend_ratio, finance.STK_XR_XD.transfer_ratio
              ).filter(finance.STK_XR_XD.code == code_hold, finance.STK_XR_XD.a_bonus_date > start_date,
                      finance.STK_XR_XD.a_bonus_date < end_date).order_by(finance.STK_XR_XD.a_bonus_date).limit(1000)
    df = finance.run_query(q)

    expand = 1
    expand_date = []
    for index, row in df.iterrows():
        #print(type(row))
        #print(row)
        if row['dividend_ratio'] and row['transfer_ratio']:
            rate = (10 + row['dividend_ratio'] + row['transfer_ratio']) / 10
            expand *= rate
        else:
            rate = (10 + (row['dividend_ratio'] if row['dividend_ratio'] is not None else 0) + (
                row['transfer_ratio'] if row['transfer_ratio'] is not None else 0) ) / 10
            expand *= rate
        
        expand_date.append([row['a_bonus_date'], rate])
    
    result = {}
    result['expand'] = expand
    result['expand_date'] = expand_date
    return result

In [5]:
'''
    output:
'''
def hold_to_buis(df1):
    
    df1 = df1.fillna(0)
    
    df = df1.diff(-1,axis=1)
    
    col_drop = df.columns[-1]
    df.drop(col_drop, axis = 1, inplace = True)
    
    return df

def align_col(df, target_col):
    
    use_col = [col for col in df.columns if col in target_col]
    
    df = df[use_col]
    return df

def expand_pure(expand):
    return expand['expand']


In [16]:
'''
    考虑复权因素，调整除权当日成交量
    注意：通过观察数据，北向持仓数量往往在“股权登记日”，也就是除权除息日的前一天就已经完成扩股
    在求得expand和成交额之后再修正，只修正需要修正的列的差值
    output:[new df, expand_rate]
'''
def revise_buis(df1):
    
    # 由于hk_hold数据从除权日前一天开始调整，因此只需要从start_date+2天开始查询拆股数据即可
    start_date = df1.columns[-3]
    end_date = df1.columns[0]
    
    df_diff = hold_to_buis(df1)
    
    df_diff['code'] = df_diff.index
    df_diff['expand'] = df_diff['code'].apply(get_expand, args = (start_date, end_date) )
    df_diff.drop('code',axis = 1, inplace=True)
    
    diff_dates = df_diff.columns.tolist()
    # 这里用简单方法，放弃apply这种抽象的方法
    for index, row in df_diff.iterrows():

        dates_change = row['expand']['expand_date']
        
        for div_date in dates_change:
            # 数据的原因，diff之后是除权的前一天交易量不对；另外一般除权日及前一天都是交易日
            # 应该是(t-1) - (t-2) * expand
            # 注意处理特殊值，例如除权日刚好是当天
            '''
                起始日除权，则不需要计算，必须跳过
                第二日除权，则数据原因，第一天的数据已经调整，也不需要计算
                第三日除权，要调整第二天的成交量，其实是diff数据的第一列，之后就正常
            '''
            tmp_div_date = div_date[0].strftime("%Y-%m-%d")
            date_idxt = diff_dates.index(tmp_div_date)
            # 要考虑diff矩阵新增expand列的影响
            ## if date_idxt < len(diff_dates) - 3:
            
            date_t1 = diff_dates[date_idxt + 1]
            date_t2 = diff_dates[date_idxt + 2]

            right_value = df1.loc[index, date_t1] - df1.loc[index, date_t2] * div_date[1]

            df_diff.loc[index, date_t1] = right_value
            
    
    df_diff['expand'] = df_diff['expand'].apply(expand_pure)
    
    # 结果最后一列为expand
    return df_diff

## 算法说明
复权的问题很重要，因为复权当日的持股数量变动一般都超过30%，价格变动也超过30%

### 聚宽数据集：
- 股价 默认为前复权，也可以不复权；
- 北向持有量 只有不复权

### 本文做法：
本文计算方法

> sum(资金净流入)/(期末持股量-期初持股量)

价格和持有量都选择不复权，计算中修正2项内容

1. 除权当日北向净成交量
2. 在结束日的视角，看到的“前复权”的持仓变动总量

## NOTES
1. join/merge/concat的区别，这里用join是不对的
2. get_price中的day不能接收非交易日，会报错，但是run_query可以，要注意
3. merge中如果遇到空的df，会全部合并为空的，要处理
4. datetime.datetime和datetime.date不同，用toordinal解决
5. iterrow中，如果要使用col_name进行索引，必须在for时加上index
6. pd.index不能使用apply
7. 常见问题就是df、series、list结构容易混乱，尤其是前两个，另外还有datetime格式与str格式的转换

In [68]:
# 
# df: hold_end_date, hold_start_date, expand
import operator

def cal_hold_change(df):
    df = df.tolist()
    #print(df)
    #print(df[0] - df[1] * df[2])
    return df[0] - df[1] * df[2]

def get_hold_change(df1, df_expand):
    
    
    
    df_tmp = df1.iloc[:,[0,-1]].merge(df_expand, left_index = True, right_index = True)
    print(df_tmp)
    result = df_tmp.apply(cal_hold_change, axis = 1)
    #print(result)
    return result


# 按位乘，按行求和
def get_net_cash(df_diff, df2):
    
    
    
    assert df_diff.shape == df2.shape, "the shape of df_diff and df2 are different"
    assert df_diff.index.tolist() == df2.index.tolist(), "the index of df_diff and df2 are different"
    assert df_diff.columns.tolist() == df2.columns.tolist(), "the columns of df_diff and df2 are different"
    
    print(df_diff)
    diff_array = df_diff.values
    df2_array = df2.values
    print(df2)
    cash_day = np.multiply(diff_array, df2_array)
    print(cash_day)
    return cash_day.sum(axis = 1)



In [66]:
## No.2 
'''
    volume_roll 过去一年北向资金成交额数据
    price_roll  过去一年收盘价
    
'''

# 初始持仓量，结束持仓量，初始持仓量调整，每日北向成交额

# import pdb
# pdb.set_trace()
import numpy as np
from jqdata import finance


def get_hold_result(end_date=None, n_days = None, securities = None):
    
    if end_date is None:
        end_date = datetime.datetime.now()
    if n_days is None:
        n_days = 180
            
#     dates = []
#     for ii in range(365):
#         dates.append((date + date.delta(-ii)).strftime("%Y-%m-%d"))
    
    # 获得结束日的北向持仓股票列表
    if securities == None:
        q0 = query(finance.STK_HK_HOLD_INFO.code).filter(finance.STK_HK_HOLD_INFO.date == date)
        df0 = finance.run_query(q0)
        
        securities = np.array(df0['code']).flatten.tolist()
    
    df1 = get_vol_roll(end_date, n_days, securities)
    print("shape of df1: ", df1.shape)
    
    # 获取开始日至收盘日之间所有日期每日的平均交易价格，注意选用不复权价格，以与每日持有量统一
    df2 = get_price_roll(end_date, n_days, securities)
    print("shape of df2: ", df2.shape)
    
    # 注意需要df3形式尽量与df2一致
    # df3 = hold_to_buis(df1)
    
    # 日期对齐，去掉无意义的日期列
    df1 = align_col(df1, df2.columns)
    print('交易日个数：', len(df1.columns))
    
    # 修正同时获得expand
    df_diff = revise_buis(df1)
    df_expand = pd.DataFrame(df_diff['expand'])
    print(type(df_expand))
    
    df_diff.drop('expand', axis = 1, inplace = True)
    
    result = pd.DataFrame({'code':df_diff.index})
    
    df_hold_change = pd.DataFrame(get_hold_change(df1, df_expand))
    df_hold_change.columns = ['hold_change_re']
    df_hold_change['code'] = df_hold_change.index
    result = result.merge(df_hold_change, on = 'code')
    # result['hold_change_re'] = 
    
    df2 = align_col(df2, df_diff.columns)
    result['net_cash'] = get_net_cash(df_diff, df2)
    
    result['avg_cost'] = result['net_cash'] / result['hold_change_re']
    
    return result



# 返回最近一年操作最终的平均持仓成本
'''
    干扰因素：拆股，全都采用前复权价格，计算中直接用总金额计算，规避干扰因素
    使用前复权可以解决分红问题，但是拆股送股，由于初始的股数并未追溯调整，所以要追溯调整股数使其准确，注意要调整的是股票数的差额
    即将起始股数调多，然后计算差额
    A股股票现金分红少见，而送转多见，所以这个调整是必须的
'''
'''
    想来想去，好像每天的成交价和成交量都不重要，成交额就可以都代表了，只需要首尾的持仓量即可
    所以是牺牲复用换取单次计算的简便
'''


'\n    想来想去，好像每天的成交价和成交量都不重要，成交额就可以都代表了，只需要首尾的持仓量即可\n    所以是牺牲复用换取单次计算的简便\n'

In [67]:
# 测试数据
# 600809.XSHG 2021-07-05 周一，1.4
# 600276.XSHG 2021-6-1, 1.2
# 601012.XSHG 2021-6-23,1.4
# 002714.XSHE 2021-6-3, 1.4

securities = ['600809.XSHG','600276.XSHG','601012.XSHG','002714.XSHE','002594.XSHE']
end_date = '2021-7-10'
n_days = 50
get_hold_result(end_date, n_days, securities)

shape of df1:  (5, 41)
shape of df2:  (5, 34)
交易日个数： 33
<class 'pandas.core.frame.DataFrame'>
             2021-07-09  2021-05-24  expand
code                                       
600809.XSHG    32386820    25541816     1.4
600276.XSHG   756180175   649609709     1.2
601012.XSHG   590129241   373829328     1.4
002714.XSHE   156089891   106274445     1.4
002594.XSHE    48030957    48438790     1.0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: 'code' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


,code,hold_change_re,net_cash,avg_cost
0,600809.XSHG,-3371722.4,-2.341772e+09,694.532762
1,600276.XSHG,-23351475.8,-2.946664e+09,126.187471
2,601012.XSHG,66768181.8,3.897863e+08,5.837904
3,002714.XSHE,7305668.0,3.117686e+08,42.674894
4,002594.XSHE,-407833.0,-2.072465e+08,508.165059


In [69]:
# 测试数据
# 600276.XSHG 2021-6-1, 1.2
# 002714.XSHE 2021-6-3, 1.4

securities = ['600276.XSHG','002714.XSHE']
end_date = '2021-6-5'
n_days = 7
get_hold_result(end_date, n_days, securities)

shape of df1:  (2, 6)
shape of df2:  (2, 5)
交易日个数： 5
<class 'pandas.core.frame.DataFrame'>
             2021-06-04  2021-05-31  expand
code                                       
600276.XSHG   637851088   661143247     1.0
002714.XSHE   151003336   112932342     1.4
             2021-06-04  2021-06-03  2021-06-02  2021-06-01
code                                                       
600276.XSHG  -5394855.0  -7395137.0 -11818010.0   1315843.0
002714.XSHE      1781.0  -1568443.0  -4040087.8  -1067995.0
             2021-06-04  2021-06-03  2021-06-02  2021-06-01
code                                                       
600276.XSHG      82.614      83.099      84.999      86.769
002714.XSHE      67.391      67.358      92.582      88.213
[[-445690550.97 -614528489.5630001 -1004519031.9899999 114174381.267]
 [120023.37100000001 -105647183.59400001 -374039408.6995983
  -94211042.93499999]]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: 'code' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version


,code,hold_change_re,net_cash,avg_cost
0,600276.XSHG,-23292159.0,-1.950564e+09,83.743362
1,002714.XSHE,-7101942.8,-5.737776e+08,80.791641


In [74]:
sum([-445690550.97,-614528489.5630001,-1004519031.9899999,114174381.267])

-1950563691.256

In [ ]:
## 分析结果

# 4种情况的各自占比，期望是负值非常少
# 将结果分别存储到不同序列种
# dict：{买入+，均价+，均价>现价；买入+，均价+，均价<现价；买入-，均价-，均价<现价；买入-，均价-，均价>现价；买入与均价异号}
# 其中情况1最佳，列出最佳排序，情况3最差，列出最差排序
# 设计函数将结果转化为因子形式


In [ ]:
def main():
    
    get_result(get_hold_price())
    
    
    # 起始日各股票HK持仓量
#     q2 = query(finance.STK_HK_HOLD_INFO.code, finance.STK_HK_HOLD_INFO.share_number
#                   ).filter(finance.STK_HK_HOLD_INFO.date == start_date, finance.STK_HK_HOLD_INFO.code.in_(securities)).limit(10000)
#     df2 = finance.run(q2)
#     df2.rename(columns={'share_number':'start_share_n'}, inplace = True)
    
#     df = df.join(df2, on = 'code')
    
   
    
#     df4 = get_price(securities, start_date=start_date, end_date=start_date, frequency='daily', 
#           fields=['close'], skip_paused=False, fq='pre', panel=False)
#     df4.rename(columns = {'close':'start_close'}, inplace = True)
#     df = df.join(df4, on = 'code')
    
#     df5 = get_price(securities, start_date= date, end_date= date, frequency='daily', 
#           fields=['close'], skip_paused=False, fq='pre', panel=False)
#     df5.rename(columns = {'close':'end_close'}, inplace = True)
#     df = df.join(df5, on = 'code')


    # dividend_ratio  每10股送XX股     transfer_ratio  每10股转增 XX股   dividend_number   transfer_number

    
# 感觉还是要引入复权因素的影响
def get_result(df_hold, df_price):
    
    assert df_hold.shape == df_price.shape
    result = df_hold.code
    
    # 追溯调整起始持仓数量，这里仍然不够精确，因为如果在送转之前卖出，那么会高估起始持仓数
    # hold_corred = correct_hold(df_hold.iloc[:,[0,1]])
    hold_corred = df_hold.iloc[:, [0,1]].apply(correct_hold, axis = 1, args = (df_hold.columns[1]))
    result['hold_change'] = df_hold.iloc[:,-1] - hold_corred
    
    # 计算期间的资金
    # 这里应该是计算每日成交额，正常应该继续调整除权除息日的持股数量，以保证当日成交股数的正确
    # 调整除权除息日初始股数
    
    # 循环相减，获得每一天的差额
    df.change = 
    
    # 计算每只股票的资金流入净额  
    df_cash = df_change.iloc[:,1:] * df_price.iloc[:,2:]
    result['df_cash_sum'] = df_cash.apply(sum, axis = 1)
    
    # 
    result['avg_buy'] = result['df_cash_sum'] / result['hold_change']
    

In [2]:
import numpy as np
from jqdata import finance

# pdb.set_trace()

secu = ['002714.XSHE','002594.XSHE']

df1 = get_price_roll(end_date = '2021-6-5',n_days=8, securities = secu)
df1

df2 = get_vol_roll(end_date = '2021-6-5',n_days=8, securities = secu)
df2

,2021-06-04,2021-06-03,2021-06-02,2021-06-01,2021-05-31
code,,,,,
002714.XSHE,67.391,67.358,92.582,88.213,88.730
002594.XSHE,187.740,183.739,187.792,185.560,175.068


### 几个问题
1. 北向的数据还是有点对不上，大致对，但总差一点
2. 测试集算出来的结果，当持续减仓时，得到的结果可能会明显大于或小于区间价格的最大最小值，值得再看看